In [1]:
import pandas as pd
import numpy as np
from scipy.fft import rfft,rfftfreq
from scipy import signal
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
def filter_signal(sig, pass_freq, stop_freq, attenuation, sample_f):
    N, Wn = signal.buttord(wp=pass_freq, ws=stop_freq, gpass=1, gstop=attenuation, analog=False, fs=sample_f)
    sos = signal.butter(N,Wn,fs=sample_f, btype="low", analog=False, output="sos")
    return signal.sosfilt(sos, sig)

In [3]:
# Create a function to downsample the dataframe using the median
def downsample_with_median(df, original_freq, target_freq):
    # Calculate the downsampling factor
    factor = original_freq // target_freq
    
    # Use the rolling window to calculate the median for each window of 'factor' rows
    downsampled_df = df.rolling(window=factor).median().iloc[factor-1::factor].reset_index(drop=True)
    
    return downsampled_df

# Assuming 'df' is the original dataframe with 3200Hz frequency
original_freq = 3200
target_freq = 200

In [ ]:
def downsample_with_mean(df, original_freq, target_freq):
    # Calculate the downsampling factor
    factor = original_freq // target_freq
    
    downsampled_df = df.rolling(window=factor).mean().iloc[factor-1::factor].reset_index(drop=True)
    
    return downsampled_df

In [ ]:
destination = Path(f"subsampled_{target_freq}_filter")
destination.mkdir(parents=True)

In [ ]:
for file in Path("labeled_dataset_typed").glob("benchy_*"):
    print(file)
    df = pd.read_parquet(file)
    df["accel_x"] = filter_signal(df["accel_x"],pass_freq,stop_freq,attenuation,fs)
    df["accel_y"] = filter_signal(df["accel_y"],pass_freq,stop_freq,attenuation,fs)
    df["accel_z"] = filter_signal(df["accel_z"],pass_freq,stop_freq,attenuation,fs)
    downsampled_df = downsample_with_median(df.drop(columns=["class"]), original_freq, target_freq)
    downsampled_df["class"] = df["class"].unique()[0]
    downsampled_df["class"] = downsampled_df["class"].astype("category")
    downsampled_df["accel_x"] = downsampled_df["accel_x"].astype("int16")
    downsampled_df["accel_y"] = downsampled_df["accel_y"].astype("int16")
    downsampled_df["accel_z"] = downsampled_df["accel_z"].astype("int16")
    downsampled_df.to_parquet(destination/file.name,compression='gzip')

In [ ]:
test = df["accel_x"][100:3300].copy()
fft_result = np.abs(rfft(test.values))
fft_freq = rfftfreq(len(test), 1/fs)

plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 2)
plt.plot(fft_freq, fft_result)
plt.title('FFT of Signal')
plt.tight_layout()
plt.show()

In [ ]:
df["accel_x"] = filter_signal(df["accel_x"],pass_freq,stop_freq,attenuation,fs)
df["accel_y"] = filter_signal(df["accel_y"],pass_freq,stop_freq,attenuation,fs)
df["accel_z"] = filter_signal(df["accel_z"],pass_freq,stop_freq,attenuation,fs)

In [ ]:
t = np.linspace(0, 3300, 3200, endpoint=False)
test2 = df["accel_y"][100:3300].copy()
fft_result = np.abs(rfft(test2.values))
fft_freq = rfftfreq(len(test2), 1/fs)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 2)
plt.plot(fft_freq, fft_result)
plt.title('FFT of Signal')
plt.tight_layout()
plt.show()